In [ ]:
! pip install openl3

  Using cached openl3-0.4.2.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327030 sha256=08f3f923b520bb7d14a4868e0bff71838a0a0b1aaa5b049bb5ed8f5bfe55b4fe
  Stored in directory: /root/.cache/pip/wheels/35/e9/4c/b1e39385b21f2b4d70c01b8793ecc921d69f167cc772868abe
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29642 sha256=eb5a7bf50c1cbf2e2ab72e8549686bb99626e26d836b8e36a2bb0529dff6a584
  Stored in directory: /root/.cache/pip/wheels/d2/17/ce/09b7d799a7c8b463b0a1657941331e0f10112864ff78d28f87
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320705 sha256=60d93f65045416f239a0b73a7569a98fb8f7beb96fdaeda1179

In [ ]:
import os
from google.colab import drive
import librosa
import soundfile as sf
import numpy as np
import openl3
import json
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imsparsh/musicnet-dataset")

100%|██████████| 21.5G/21.5G [16:23<00:00, 23.5MB/s]

Extracting files...


In [ ]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1


In [ ]:
wav_dir = os.path.join(path, "musicnet")
print("WAV目录:", wav_dir)

WAV目录: /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet


In [ ]:
OUTPUT_EMBEDDING_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_embeddings"
METADATA_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_metadata"

os.makedirs(OUTPUT_EMBEDDING_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

In [ ]:
model = openl3.models.load_audio_embedding_model(
    input_repr="mel256",
    content_type="music",
    embedding_size=6144
)

In [ ]:
def segment_audio(audio, sr, segment_length=10.0, stride=5.0):
    # 切音频，10s一段，5秒overlap
    segments = []
    metadata = []
    # 存分段和描述信息
    total_duration = len(audio) / sr
    start = 0.0
    segment_id = 0
    while start < total_duration:
        end = start + segment_length
        start_sample = int(start * sr)
        end_sample = int(end * sr)
        if end_sample > len(audio):
            segment = np.pad(audio[start_sample:], (0, end_sample - len(audio)))
            is_padded = True
        # 处理音频结尾分段超过整首歌长度
        else:
            segment = audio[start_sample:end_sample]
            is_padded = False
        segments.append(segment)
        # 加入分段
        metadata.append({
            "segment_id": segment_id,
            "start_time": float(start),
            "end_time": float(min(end, total_duration)),
            "duration": segment_length,
            "is_padded": is_padded,
            "overlap_previous": start > 0,
            "overlap_next": (end < total_duration),
        })
        # 加入分段信息
        start += stride
        segment_id += 1
    return segments, metadata

In [ ]:
for root, dirs, files in os.walk(wav_dir):
    for fname in tqdm(files, desc=f"正在处理目录 {root}"):
        if fname.endswith(".wav"):
            wav_path = os.path.join(root, fname)
            recording_id = os.path.splitext(fname)[0]
            audio, sr = librosa.load(wav_path, sr=48000)
            # 调整采样率

            segments, seg_metadata = segment_audio(audio, sr)
            # 拆成10s一段

            with open(os.path.join(METADATA_DIR, f"{recording_id}_metadata.json"), "w") as f:
                json.dump(seg_metadata, f, indent=2)
            # 存分段信息

            for seg_id, segment in enumerate(segments):
                # 遍历，提取embedding
                emb, _ = openl3.get_audio_embedding(
                    segment,
                    sr,
                    model=model,
                    center=False,
                    hop_size=1.0,
                    verbose=False
                )
                aggregated_emb = np.mean(emb, axis=0)
                # 把每秒1个的10个向量合成一个
                np.save(
                    os.path.join(OUTPUT_EMBEDDING_DIR, f"{recording_id}_seg{seg_id}.npy"),
                    aggregated_emb
                )
                # 存数据


正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet: 0it [00:00, ?it/s]
正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet/musicnet: 0it [00:00, ?it/s]
正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet/musicnet/test_labels: 100%|██████████| 10/10 [00:00<00:00, 108100.62it/s]
正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet/musicnet/test_data: 100%|██████████| 10/10 [03:50<00:00, 23.02s/it]
正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet/musicnet/train_data: 100%|██████████| 320/320 [51:03<00:00,  9.57s/it]
正在处理目录 /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1/musicnet/musicnet/train_labels: 100%|██████████| 320/320 [00:00<00:00, 1342177.28it/s]
